In [1]:
### This file contains code to collect data from InPhO ###
#The entries to be downloaded will be those entries that have not been already downloaded through SEP.

In [2]:
import pymongo
import json
import requests
import time

from tqdm import tqdm
from pprint import pprint

import lib_sepinpho as sep


In [3]:
##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#connect to database
db = client.visualizing_sep
sep_collection = db.sep_entries
inpho_collection = db.inpho_data

In [4]:
#get list of InPhO ids already in SEP. 
#We use this to get a list of what we DONT need to download from InPhO again. 
inpho_ids_in_SEP = inpho_collection.distinct('ID')

#get distinct lists for each of the major keys from the collection
links = sorted(inpho_collection.distinct('links'))
related = sorted(inpho_collection.distinct('related'))
instances = sorted(inpho_collection.distinct('instances'))
classes = sorted(inpho_collection.distinct('classes'))
hyponyms = sorted(inpho_collection.distinct('hyponyms'))
nodes = sorted(inpho_collection.distinct('nodes'))
occurences = sorted(inpho_collection.distinct('occurences'))
related_thinkers = sorted(inpho_collection.distinct('related_thinkers'))
related_ideas = sorted(inpho_collection.distinct('related_ideas'))
influenced = sorted(inpho_collection.distinct('influenced'))
influenced_by = sorted(inpho_collection.distinct('influenced_by'))
teachers = sorted(inpho_collection.distinct('teachers'))
students = sorted(inpho_collection.distinct('students'))

#get counts of keys
print(f"links: {len(links)}")
print(f"related: {len(related)}")
print(f"instances: {len(instances)}")
print(f"classes: {len(classes)}")
print(f"hyponyms: {len(hyponyms)}")
print(f"nodes: {len(nodes)}")
print(f"related_thinkers: {len(related_thinkers)}")
print(f"related_ideas: {len(related_ideas)}")
print(f"influenced: {len(influenced)}")
print(f"influenced_by: {len(influenced_by)}")
print(f"teachers: {len(teachers)}")
print(f"students: {len(students)}")

#get unique list of all ids in the collection
inpho_ids_all = set(links + related + instances + hyponyms + nodes + related_ideas\
                          + related_thinkers + influenced + influenced_by\
                          + teachers + students)

#get list of ids to download that are NOT in inpho_ids_in_SEP
inpho_ids_toget = [id for id in inpho_ids_all if id not in inpho_ids_in_SEP]
inpho_ids_toget = sorted(inpho_ids_toget)

print(inpho_ids_toget)

links: 71
related: 863
instances: 337
classes: 0
hyponyms: 467
nodes: 113
related_thinkers: 435
related_ideas: 629
influenced: 438
influenced_by: 360
teachers: 94
students: 149
[]


In [29]:
### Download JSON data for InPho Entities###
### Date Completed: 4.11.2010 ###

for id_val in tqdm(inpho_ids_toget):
    #create valid api url
    api_url = f'http://inpho.cogs.indiana.edu/entity/{id_val}.json'
    try:
        json_data = sep.return_json_data(api_url)
        inpho_collection.insert_one(json_data) 
    except:
        print('Failed @: {api_url}')
    time.sleep(3)
    


9
100%|██████████| 9/9 [00:31<00:00,  3.49s/it]


In [39]:
inpho_ids_in_SEP = sep_collection.distinct('inpho_json.ID')
inpho_ids_in_SEP_response = sep_collection.distinct('inpho_json.responseData.result.ID')
inpho_ids_in_INPHO = inpho_collection.distinct('ID')
print(1585 + 91)
print(len(inpho_ids_in_SEP))
print(len(inpho_ids_in_SEP_response))
print(len(inpho_ids_in_INPHO))

1676
1585
91
2330
